In [ ]:
import numpy as np
import os
import hashlib
import tensorflow as tf
from PIL import Image
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Correct import
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from os import listdir

In [ ]:
# Check for TPU
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
except ValueError:
    strategy = tf.distribute.get_strategy()  # Default to CPU/GPU strategy if TPU is not found


In [ ]:

def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)  # Randomly flip images
    image = tf.image.random_brightness(image, max_delta=0.1)  # Random brightness
    return image, label

train_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/ai-vs-real-dataset/_DATASET/dataset/train',
    image_size=(224, 224),
    batch_size=128,  # Adjust based on your TPU memory
    shuffle=True
)
    
train_data = train_data.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
    # Prefetch to improve performance
train_data = train_data.prefetch(tf.data.AUTOTUNE)

In [ ]:

with strategy.scope():
    base_model = tf.keras.applications.EfficientNetB0(weights='/kaggle/input/efficientnetb0-weights/efficientnetb0_notop.h5', include_top=False, input_shape=(224, 224, 3))
    
    # Add custom classification layers
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    # Create the final model
    model = tf.keras.Model(inputs=base_model.input, outputs=x)

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


In [ ]:
with strategy.scope():
    model.fit(train_data, epochs=20)
    model.save('/kaggle/working/EfficientNet_pre_model.h5')

In [ ]:
# # Test images here

# Load the test dataset
test_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/ai-vs-real-dataset/_DATASET/dataset/test',  # Your test directory
    image_size=(224, 224),
    batch_size=64,
    shuffle=True  # Important to keep the order for predictions
)


# Extract true labels from the dataset
y_true = np.concatenate([y.numpy() for x, y in test_data], axis=0)
class_names = ['FAKE', 'REAL']

# Make predictions
with strategy.scope():
    predictions = model.predict(test_data)

# Convert predictions to binary labels
y_pred = (predictions > 0.5).astype("int32").flatten()  # Flatten to match shape of y_true

# Create confusion matrix
cm = confusion_matrix(y_true, y_pred)

print("Confusion Matrix:")
print(cm)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Plot confusion matrix for better visualization
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Unfreeze top layers for fine-tuning
with strategy.scope():
    base_model.trainable = True

# Fine-tune from this layer onwards (adjust as needed)
    for layer in base_model.layers[:150]:
        layer.trainable = False

# Re-compile the model with a lower learning rate
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

# Train again

    history_fine = model.fit(
        train_data,
        epochs=10  # Fine-tuning for a few additional epochs
    )
    model.save('/kaggle/working/EfficientNet_fine_tune_model.h5')



In [ ]:

test_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/ai-vs-real-dataset/_DATASET/dataset/test', 
    image_size=(224, 224),
    batch_size=64,
    shuffle=True  
)

# Extract true labels from the dataset
y_true = np.concatenate([y.numpy() for x, y in test_data], axis=0)
class_names = ['FAKE', 'REAL']

# Make predictions
with strategy.scope():
    predictions = model.predict(test_data)

# Convert predictions to binary labels
y_pred = (predictions > 0.5).astype("int32").flatten()  # Flatten to match shape of y_true

# Create confusion matrix
cm = confusion_matrix(y_true, y_pred)

print("Confusion Matrix:")
print(cm)

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Plot confusion matrix for better visualization
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
model = tf.keras.models.load_model('/kaggle/working/EfficientNet_fine_tune_model.h5')
accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy[1]}")


In [ ]:
#for deployment
def predictPicture(imagePath):
    # model=tf.keras.models.load_model('//kaggle/input/efficientnetmodel-5/EfficientNet_model_5/saved_model.pb')
    if imagePath.endswith((".png", ".jpg", ".jpeg")):
        # Load and preprocess the image
        img = load_img(imagePath, target_size=(224, 224))  # Resize image to model input size
        img_arr = img_to_array(img)  # Convert to array
        img_arr = np.expand_dims(img_arr, axis=0) # Add batch dimension
    
        with strategy.scope():
            prediction = model.predict(img_arr)
        
        # Output the predicted probability
        print(prediction[0])
        
        # Output the label based on probability
        if prediction[0] > 0.5:
            print("Prediction: Real")
        else:
            print("Prediction: Fake")
    else:
        print("Please provide a valid image file (PNG or JPG format).")
# get the path or directory
fake_test = "/kaggle/input/ai-vs-real-dataset/_DATASET/dataset/test/FAKE"
i=0
for images in os.listdir(fake_test):
    file_path=fake_test+"/"+images
    if i%200==0:
        predictPicture(file_path)
    i+=1
i=0
print("CHANGE")
real_test = "/kaggle/input/ai-vs-real-dataset/_DATASET/dataset/test/REAL"
for images in os.listdir(real_test):
    file_path=real_test+"/"+images
    if i%200==0:
        predictPicture(file_path)    
    i+=1